In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

In [3]:
data = pd.read_csv('./train.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


In [5]:
data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [6]:
data = data.loc[data["Rooms"]<7, :]
data.shape

(9997, 20)

In [7]:
data = data.loc[data["Price"].between(30000,600000), :]
data.shape

(9977, 20)

In [8]:
data.loc[:,"DistrictId"].unique()

array([ 35,  41,  53,  58,  99,  59, 154,  74,   1,  23,  28,  31,  13,
        57,  27,  85,  34,  19,   0,  48,  54, 150,  11,  50,  61,  44,
        90, 200,  43, 177, 101,  38,  21, 103,  98, 170,  94,   5,  88,
        47,   6,  52,  62,  77,  79,   9, 156,  69, 108,  96, 169,  18,
       148, 128,  10, 144, 120,  39,   7,  84, 123, 159,  70, 129,   3,
        95,  78,  45,  15, 121,  49,  25,   2, 127,  56, 109,  46,  17,
       119,  80,  86,  83, 137,  91,  92, 118, 187,  36, 106, 161,  22,
        63, 167, 113,  66,  29,  37,  20,   8, 181,  40,  93,  30,  81,
       107, 130, 100,  12, 125,  67,  26,  89,  24, 111, 138,  64, 124,
       183,  55, 157, 151,  33,  87, 126, 155, 164, 105, 110, 132,  71,
        73, 189,  75, 122, 163, 141,  76, 115, 139, 143, 201, 165, 190,
       193, 174,  60, 114, 172, 134, 117, 149, 153, 131, 145,  82, 160,
        42,  32, 133, 208, 171, 176, 173, 147,  97, 142, 195, 180, 196,
       112,  72, 175,   4, 186, 146, 191, 168, 192,  14, 199, 13

In [9]:
district_mean_price=pd.DataFrame({'DistrictId': data['DistrictId'].unique(),
                          'district_mean_price': [data.loc[data['DistrictId'] == i, 'Price'].mean() for i in data['DistrictId'].unique()]})

In [10]:
data = pd.merge(data,district_mean_price, on = 'DistrictId', how = 'outer')

In [11]:
data.loc[data["LifeSquare"].isnull(),'LifeSquare'] = data['Square']*0.82

In [12]:
data = data.loc[data["Square"]>16, :]
data.shape

(9964, 21)

In [13]:
square_mean_1 = data.loc[data['Rooms']<=1, 'Square'].mean()
square_mean_3 = data.loc[data['Rooms']==3, 'Square'].mean()

In [14]:
data.loc[(data['Square'] < 15) & (data['LifeSquare'] < 15) & (data['Rooms']<=1), 'Square'] = square_mean_1

data.loc[(data['Square'] < 15) & (data['LifeSquare'] < 15) & (data['Rooms']==3), 'Square'] = square_mean_3

data.loc[(data['Square'] > 15) & (data['LifeSquare'] < 15), 'LifeSquare'] = data['Square']

data.loc[data['Square'] < data['LifeSquare'], 'LifeSquare'] = (data['Square']+data['Square']*0.82)

In [15]:
data = pd.get_dummies(data)

In [16]:
data = data.drop('Healthcare_1', axis=1)

In [17]:
train, valid = train_test_split(data, test_size=0.3, random_state=666)

In [18]:
model = RandomForestRegressor(n_estimators=1000, max_depth=15, random_state=666, min_samples_leaf=2,n_jobs=-1)

In [19]:
fts = ['Rooms', 'Square', 'LifeSquare',
       'HouseYear', 'Ecology_1', 'Social_1', 'Social_2',
       'Social_3', 'Helthcare_2', 'Shops_1', 'Ecology_2_A',
       'Ecology_2_B', 'Ecology_3_A', 'Ecology_3_B', 'Shops_2_A', 'Shops_2_B','district_mean_price']

In [20]:
model.fit(train.loc[:,fts], train['Price'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=False, random_state=666, verbose=0, warm_start=False)

In [21]:
pred = model.predict(train.loc[:,fts])

In [22]:
r2_score(train['Price'],pred)

0.9249410126122298

In [23]:
pred_valid = model.predict(valid.loc[:,fts])

In [24]:
r2_score(valid['Price'],pred_valid)

0.7434110218438185

In [25]:
test = pd.read_csv('./test.csv')

In [26]:
test = test.drop('Healthcare_1', axis=1)

In [27]:
test.loc[(test['Square'] < 15) & (test['LifeSquare'] < 15) & (data['Rooms']<=1), 'Square'] = square_mean_1

test.loc[(test['Square'] < 15) & (test['LifeSquare'] < 15) & (data['Rooms']==3), 'Square'] = square_mean_3

test.loc[(test['Square'] > 15) & (test['LifeSquare'] < 15), 'LifeSquare'] = test['Square']

test.loc[test['Square'] < test['LifeSquare'], 'LifeSquare'] = (test['Square']+test['Square']*0.82)

In [28]:
test = pd.get_dummies(test)

In [29]:
test.loc[test["LifeSquare"].isnull(),'LifeSquare'] = test['Square']*0.82

In [30]:
district_mean_price=pd.DataFrame({'DistrictId': test['DistrictId'].unique(),
                          'district_mean_price': [data.loc[data['DistrictId'] == i, 'Price'].mean() for i in test['DistrictId'].unique()]})
district_mean_price_mean = district_mean_price['district_mean_price'].mean()

In [31]:
test = pd.merge(test,district_mean_price, on = 'DistrictId', how = 'outer')

In [32]:
test.loc[test["district_mean_price"].isnull(),'district_mean_price'] = district_mean_price_mean

In [33]:
pred_test = model.predict(test.loc[:, fts])

In [34]:
test['Price'] = pred_test

In [35]:
SGoryushko_predictions = test.loc[:, ['Id', 'Price']]

In [36]:
SGoryushko_predictions.to_csv('SGoryushko_predictions.csv', index=None)